# Run Galfit on COSMOS galaxies
- 10/27/2020: Updated to use galfit_helpers.py module

### Load modules

In [1]:
from __future__ import division
import sys
sys.path.insert(0,'/data/emiln/XLSSC122_GalPops/Analysis/Modules')
from GalfitPyWrap import galfitwrap as gf
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import glob
from scipy import ndimage
from astropy import units as u
from astropy import wcs
from astropy.coordinates import SkyCoord
from astropy.io import fits, ascii
from astropy.table import Table, hstack, join
import pickle
import fnmatch
import multiprocessing as mp
%matplotlib inline
%load_ext autoreload
%autoreload 2

/data/emiln/miniconda2/lib/python2.7/site-packages/pyfits/__init__.py:22: PyFITSDeprecationWarning: PyFITS is deprecated, please use astropy.io.fits
  PyFITSDeprecationWarning)  # noqa


In [2]:
import galfit_helpers as gfh

### Load COSMOS galaxy catalog

In [3]:
COSMOS_cat_root = '/data/emiln/XLSSC122_GalPops/Data/Products/COSMOS/catalogs/'
COSMOS_cat_filename = COSMOS_cat_root+'cos_df_all_merged.csv'
full_df = gfh.load_COSMOS_galaxy_catalog(COSMOS_cat_filename,mag='F125W_Kron',magthresh=30, overwrite=False, verbose=True)
filtered_df = gfh.load_COSMOS_galaxy_catalog(COSMOS_cat_filename,magthresh=24, mag='F125W_Kron',overwrite=False, verbose=True)


Checking if catalog exists at /data/emiln/XLSSC122_GalPops/Data/COSMOS/Products/catalogs/magthresh30.csv
Catalog already exists, returning DataFrame
Checking if catalog exists at /data/emiln/XLSSC122_GalPops/Data/COSMOS/Products/catalogs/magthresh24.csv
Catalog already exists, returning DataFrame
/data/emiln/miniconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2818: DtypeWarning: Columns (399) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [25]:
z2_filtered_df = gfh.load_COSMOS_galaxy_catalog(COSMOS_cat_filename,magthresh=24, z=1.98,z_thresh=0.1,overwrite=False,verbose=True) 

Checking if catalog exists at /data/emiln/XLSSC122_GalPops/Data/COSMOS/Products/catalogs/magthresh24_z1.98_zthresh0.1.csv
Catalog already exists, returning DataFrame


In [26]:
print len(z2_filtered_df)

156


### Set Galfit parameters

In [60]:
# fit_df = filtered_df[0:4]
fit_df = z2_filtered_df[0:8]
df_name = 'TEST_modular_galfit'
psf_root = '/data/emiln/XLSSC122_GalPops/Data/Products/COSMOS/PSF/'
psf_file = psf_root+'COSMOS/psfJ.fits' # J = F125W, H=F160W
PSFf = 1 
convbox='40 40'
image_width = 400 # 200 for mag<23 objects, 400 for z2 objects
timeout=2*60
verbose=True
width=10
HLRwidth=10 # Cutout width = 2*HLwidth
PA_INIT = 45
AR_INIT = 0.5
# zp = -21.1 # From header
ZP = 26.2303 # From https://archive.stsci.edu/pub/hlsp/candels/cosmos/cos-tot/v1.0/hlsp_candels_hst_cos-tot_readme_v1.0.pdf
MAG_INIT = 21
fitMagnitude = True
neighbourMagThresh=2
DYNMAG = True
sky='Default'
sky_INIT=0.00001
sigma_file = '/sigma_rms_meanexp_cps.fits'
data_file = '/data_cps.fits'
constraint_file = 'galfit_constr.txt' # Matches van der wel 2012 constraints
# constr_file = 'none'
badmask='none'
save_name = None

In [61]:
params = {
    'fit_df':fit_df, # Dataframe with objects to be fit
    'full_df':full_df, # Unfiltered source catalog used for fitting neighbours
    'width':width, # Fitting region width in pixels
    'HLRwidth':HLRwidth, # Fitting region width in # of HLR
    'sigma_file':sigma_file, # Filename of sigma maps for sources
    'data_file':data_file, # Filename for data cutouts for each source
    'PSF_file':psf_file, # File_name of PSF to be used
    'usePSF':True, # Use PSF in fitting?
    'timeout':timeout, # Max runtime per object in seconds
    'PSFf':PSFf, # Fine sampling factor
    'verbose':verbose, # Verbose mode
    'PA_INIT':PA_INIT, # Initial position angle
    'AR_INIT':AR_INIT, # Initial axis ratio
    'MAG_INIT':MAG_INIT, # Initial magnitude
    'convbox':convbox, # Region to convolve PSF with in pixels (i.e. '100 100')
    'ZP':ZP, # Zeropoint 
    'constraint_file':constraint_file, # Galfit constraint filename
    'image_width':image_width, # Size of data+sigma images being used (200 for COSMOS cutouts)
    'useDYNMAG':DYNMAG, # Initialize magnitudes from catalog?
    'badmask':badmask, # filename for bad pixel mask
    'fitMagnitude':fitMagnitude, # Fit magnitudes?
    'sky':sky, # Sky fitting mode for galfit (i.e. 'default')
    'sky_INIT':sky_INIT, # Initial sky level
    'neighbourMagThresh':neighbourMagThresh, # Additional magnitude threshhold to fit neighbours (i.e. 3 -> only neighbours with mag < source_mag+3 are fit)
    'df_name': df_name, # Descriptive name of catalog being fit
    'save_name':save_name # Filename to save results to, overrides default
}

### Run Galfit

In [62]:
gfh.run_galfit_parallel(params,**params) # Feeds in unwrapped params as well as a copy to be passed as kwargs to sub functions

Initializing 185 with F125W Kron magnitude: 23.52
ID 185
RA: 150.0730022
DEC: 2.1812181
Initial X: 9997.76290834
Initial Y: 8015.23773933
Cutout X: 400
Cutout Y: 400
Cutout width: 400
Cutoutwidth (pixels) for ID 185 : 96
Cutoutwidth (arcsec) for ID 185 : 5.76
Bounds: [352, 448, 352, 448]
0 NEIGHBOURS FOUND
Adding additional model components for neighbours...
Using PSF
Running galfit from CWD: /data/emiln/XLSSC122_GalPops/Data/Products/COSMOS/galfit_results/185
Initializing 453 with F125W Kron magnitude: 22.36
ID 453
RA: 150.1254026
DEC: 2.185425
Initial X: 6856.01840173
Initial Y: 8267.61598913
Cutout X: 400
Cutout Y: 400
Cutout width: 400
Cutoutwidth (pixels) for ID 453 : 84
Cutoutwidth (arcsec) for ID 453 : 5.04
Bounds: [358, 442, 358, 442]
0 NEIGHBOURS FOUND
Adding additional model components for neighbours...
Using PSF
Running galfit from CWD: /data/emiln/XLSSC122_GalPops/Data/Products/COSMOS/galfit_results/453
SAVE_NAME:TEST_modular_galfit_data_cps_sigma_rms_meanexp_cps_w10HLR_psf

,ID,ra,dec,re,ar,n,mag,sky,chi2nu,ErrorValue
0,185,150.073002,2.181218,4.1045,0.4733,0.9058,23.2663,0.000560,0.532096,0
1,453,150.125403,2.185425,3.3696,0.4411,1.8130,22.0147,-0.000856,0.195907,0
2,801,150.197511,2.190162,2.5864,0.7202,1.4828,23.2504,-0.000959,0.202377,0
3,854,150.139466,2.190714,1.4867,0.6486,2.8059,23.0647,-0.000482,0.186977,0
4,864,150.190505,2.190650,6.0023,0.4881,0.8317,23.2182,-0.000840,0.185392,0
5,998,150.166715,2.191634,9.7573,0.5667,1.4572,22.5496,-0.000592,0.789280,0
6,1359,150.162772,2.196018,2.4612,0.7199,4.2442,22.4550,-0.000565,0.280147,0
7,1918,150.138919,2.202854,5.0732,0.3107,0.6523,23.6752,-0.000497,0.215250,0


### Validate results

In [ ]:
# Plot fits by ID

In [ ]:
# 

### Save results